## Ant Colony Optimization - Monte Carol Hybrid for the Set Covering Problem

Set of files used for testing found in the resources folder of the project (./resources)

In [123]:
import numpy as np



Definition of the problem class which will contain the data and methods related to a problem. Each object will be an instance of a problem that will be used by each ant.

In [124]:
class Problem:
    
    # Definition of problem variables to be used in the solution
    num_elements = 0
    num_sets = 0
    sets_cost = []
    matrix = None
    
    covered_sets = []
    uncovered_sets = []
    
    covered_elements = []
    uncovered_elements = []
    
    
    # Helper function to add flag elements to the reference matrix
    def addElements(self, mat, idx, elems):
        for e in elems:
            mat[idx,(e-1)] = 1
        return mat

    # Definition of the function in charge to read the instance file.
    def read_file(self, fname):
        with open(fname) as f:
            content = f.readlines()
        self.num_elements, self.num_sets = content[0].split()
        self.num_elements = int(self.num_elements)
        self.num_sets = int(self.num_sets)

        self.matrix = np.zeros((self.num_elements,self.num_sets), dtype=np.int)

        line_idx = 1

        while(len(self.sets_cost) < num_sets):
            self.sets_cost.extend([int(x) for x in content[line_idx].split()])
            line_idx += 1

        for r in range(0, num_elements):
            tmp_list=[]
            n_els = int(content[line_idx])
            line_idx += 1
            while(len(tmp_list) < n_els):
                tmp_list.extend([int(x) for x in content[line_idx].split()])
                line_idx += 1
            self.matrix = self.addElements(self.matrix, r, tmp_list)
            
        #Initialize required values after data load
        self.init_covered_lists()
    
    def init_covered_elements(self):
        self.covered_elements = []
        self.uncovered_elements = list(range(0, self.num_elements))
    
    def init_covered_sets(self):
        covered_sets = []
        uncovered_sets = list(range(0, self.num_sets))
    
    def init_covered_lists(self):
        self.init_covered_elements()
        self.init_covered_sets()
        
    
    def covered_sets_cost(self):
        cov_cost = 0
        for cs in self.covered_sets:
            cov_cost += sets_cost[cs]
        return cov_cost

In [125]:
class ProblemSolver:
    # Properties used for the solution
    problem = None
    ants = None
    pheromone = None
    best_ant = None
    min_pheromone = None
    max_pheromone = None
    number_of_ants = 40
    rho = 0.8
    epsilon = 0.01
    beta = 5.0
    
    def __init__(self, instance_file):
        self.problem = Problem()
        self.problem.read_file(instance_file)
        self.init_pheromone()
        self.init_ants(self.init_heuristic_information(), self.beta)
    
    # Method to initialize the pheromone values
    def init_pheromone(self):
        self.pheromone = np.empty(self.problem.num_sets)
        self.pheromone.fill(9999999)
        self.max_pheromone = 9999999
    
    # Initialize the array of ants with the values for the 
    # problem instance, pheromone, heuristic information and beta
    def init_ants(self,heuristic_info, b):
        self.ants = []
        for i in range(0,self.number_of_ants):
            self.ants.append(Ant(self.problem,self.pheromone, heuristic_info, b))
    
    # Initially the heuristic information for each set has the value of the 
    # number of elements divided by the cost of the set
    def init_heuristic_information(self):
        heuristic_info = []
        for i in range(0, self.problem.num_sets):
            heuristic_info.append(np.sum(self.problem.matrix[:,i])/self.problem.sets_cost[i])
        return heuristic_info
    
    def solve(self, instance_location):
        first_loop = True
        loop_counter = 0
        while not self.terminate(loop_counter):
            current_best_ant = self.best_ant
            for ant in self.ants:
                ant.solve()
                # Apply redundancy elimination here
                if current_best_ant is None:
                    current_best_ant = ant
                if best_ant is None or (best_ant is not None and best_ant.problem.covered_sets_cost()):
                    best_ant = Ant(ant.problem, ant.pheromone, ant.heuristic_information, ant.beta)
        
    def terminate(self, loop_count):
        return loop_count >= max_loops
    

Definition of the Ant class. Ant's are the basic unit of an Ant Colony Optimization algorithm. In this case the ants contain three main data objects. One to store an instance of the problem. Another one to store the pheromone information and a last one for the heuristic information. In this case a variation of **Max-Min Ant System** is used. There is an important concept for these kind of algorithms known as the pheromone. This idea gives the colony a global communication method, so that ants can share information about their independent process and make the others aware of their findings, contributing to the general global objective.

In this solution the heurstic and pheromone information are defined around the sets, this is a key point that allows the use of an Ant Colony Optimization problem definition for the set covering problem, given that it is not very straightforward to find the conditions that allow using this algorithm for any kind of problem. Here each ant has to select of a set to cover in each step giving the idea of a path followed by the ant. The method introduces monte carlo perturbations to set the probability of each set after every iteration over the colony.

**WHAT IS A PERTURBATION IN A MARKOV CHAIN MONTE CARLO METHOD????**

**ADD PHEROMONE CALCULATION FORMULA HERE** Try to find something implemented for MCMC in ipython and just modify the latex code for the Sum and division.

The main idea behind the basic definition of Max-Min Ant System is to define an upper and lower limit for the pheromone values to give the algorithm a higher chance to escape from local minima.

In [126]:
class Ant:
    
    # Main properties of the class
    problem = Problem()
    pheromone = None
    heuristic_information = None
    
    # MAKE SURE THIS PARAMETER IS GOING TO BE USED. SUPOSEDLY USED IN THE PROBABILITY TO PICK A SET IN THE CONSTRUCTION
    beta = None
    
    def __init__(self, prob, pher, heur_inf, b):
        self.problem = prob
        self.pheromone = pher
        self.heuristic_information = heur_inf
        self.beta = b

In [127]:
prob =  Problem()

solver =  ProblemSolver("./resources/scp-instances/scp42.txt")